### Histogram 1D

In [1]:
import numpy as np
import time
import asyncio as aio
import pandas as pd
import ipywidgets as widgets
from vega.widget import VegaWidget
nbins = 50
spec_no_data = {
    '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
     'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
     'data': {'name': 'data'},
     'encoding': {
                  'x': {'field': 'x', 'type': 'ordinal'},
                  'y': {'field': 'y', 'type': 'quantitative'}
             },
     'mark': 'bar'}
spec_with_data = spec_no_data.copy()
nsamples = 100_000
steps = 100
rdata = np.random.normal(loc=0, scale=0.1, size=nsamples)
widget = VegaWidget(spec=spec_no_data)
progress = widgets.IntProgress(value=0, min=0, max=steps, description='progress', orientation='horizontal')
display(widgets.VBox([widget, progress]))
async def progressive_loop(widget, nbins=50):
    hist = np.zeros(nbins)
    step = 0
    for sl in np.split(rdata, steps):
        progress.value = step
        step += 1
        await aio.sleep(0.1)
        h, _ = np.histogram(sl, bins=nbins, range=(-1.0,1.0))
        hist += h
        source = pd.DataFrame({
            'x': range(nbins),
            'y': hist,
        }, dtype='float32')
        #  widget.update('data', remove='datum.x>=0')
        #print("displayed", widget._displayed)
        widget._displayed = True
        widget.update('data', insert=source, remove="true")

await progressive_loop(widget)

### Histogram 2D

In [ ]:
import time
import numpy as np
import asyncio as aio
import pandas as pd
from vega.widget import VegaWidget
from ipytablewidgets import LZ4Compressor
import ipywidgets as widgets
nbins = 256
spec_no_data = {'$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
 "width": 500,
 "height": 400,
 'data': {'name': 'data'},
 'encoding': {'color': {'field': 'z', 
                        'type': 'quantitative',
                        # "scale": {
                        # "domain": [0,1]  # Remove if domain changes
                        # }
                        },
  'x': {'field': 'x', 'type': 'ordinal'},
  'y': {'field': 'y', 'type': 'ordinal'}},
 'mark': 'rect',
 "config": {
    "axis": {
      "disable": True  # Change to False to see the ticks
    }
  }}
widget = VegaWidget(spec=spec_no_data)
widget.compression = LZ4Compressor(2)
steps = 100
progress = widgets.IntProgress(value=0, min=0, max=steps, description='progress', orientation='horizontal')
mean = widgets.Text(value="", description='Mean time', disables=True)
std = widgets.Text(value="",  description='Std time  ', disables=True)
display(widgets.VBox([widget, progress, mean, std]))
nsamples=100_000
means = [0.1, 0.3]
cov = [[0.01, 0], [0, 0.09]]
rdata = np.random.multivariate_normal(means, cov, size=(nsamples))
async def progressive_loop(widget, nbins=nbins):
    hist = np.zeros((nbins,nbins), dtype='uint32')
    step = 0
    times = []
    for sl in np.split(rdata, steps):
        progress.value = step
        step += 1
        await aio.sleep(0)
        h, *_ = np.histogram2d(sl[:,0],sl[:,1], bins=nbins, range=((-1.0,1.0),(-1.0,1.0)))
        hist += h.astype('uint32')
        #print(hist.shape)
        widget._displayed = True
        # max = np.max(hist)
        # value = hist/max
        value = hist
        widget._displayed = True
        start = time.time()
        widget.update('data', insert=value, remove="true")
        times.append(time.time() - start)
        mean.value = f"{np.mean(times)*1000:.3f}ms"
        std.value = f"{np.std(times)*1000:.3f}ms"
        
await progressive_loop(widget, nbins)